# Load Datasets

In [1]:
import numpy as np
import pandas as pd
import os

import sklearn.tree
import sklearn.ensemble

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [2]:
# Clone repository in order to get access locally to the datasets
!rm -rf .git README.md
!git clone -b playing-emsembled-methods https://github.com/sergio-gimenez/anomaly-4G-detection 

fatal: destination path 'anomaly-4G-detection' already exists and is not an empty directory.


In [3]:
!ls -la

total 776
drwxr-xr-x 1 root root   4096 Dec 30 17:37 .
drwxr-xr-x 1 root root   4096 Dec 30 14:43 ..
drwxr-xr-x 3 root root   4096 Dec 30 15:24 a
drwxr-xr-x 4 root root   4096 Dec 30 17:37 anomaly-4G-detection
drwxr-xr-x 4 root root   4096 Dec 30 15:21 anomaly-4G-detectiona
drwxr-xr-x 3 root root   4096 Dec 30 15:22 anomaly-4G-detectionsd
drwxr-xr-x 1 root root   4096 Dec 21 17:29 .config
drwxr-xr-x 3 root root   4096 Dec 30 15:22 ds
drwxr-xr-x 2 root root   4096 Dec 30 15:20 .ipynb_checkpoints
drwxr-xr-x 1 root root   4096 Dec 21 17:29 sample_data
-rw-r--r-- 1 root root 751186 Dec 30 17:35 voting_model.joblib


In [4]:
train = pd.read_csv('anomaly-4G-detection/ML-MATT-CompetitionQT2021_train.csv', sep=';')
test = pd.read_csv('anomaly-4G-detection/ML-MATT-CompetitionQT2021_test.xls', sep=';' )

In [5]:
# Separate labels from data 
X = train.drop('Unusual', axis='columns')#.to_numpy()
y = train['Unusual']#.to_numpy()

# We split the data into training and validation subsets (80% and 20%) in
# order to validate our training
X_train, X_validation, y_train, y_validation = train_test_split(X, y, 
                                                                train_size=0.8,
                                                                random_state=1, stratify = y)
X_test = test

In [6]:
#Refactor time feature to minuts and cellName to unique identifier 1:1
def getTimeInMinutes(x):
  hh, mm  = x.split(":")
  return int(hh)* 60 + int(mm)

def createCellNameDictionary(data):
  cellList = []
  for i in data["CellName"]:
    cellList.append(i)
  cellList = set(cellList)
  cellDict = {}
  for idx, value in enumerate(cellList):
    cellDict[value]=idx
  return cellDict

def refactorFeaturesDataframe(data):
  data["Time"] = data["Time"].apply(lambda x: getTimeInMinutes(x))
  cellNameDict = createCellNameDictionary(data);
  data["CellName"] = data["CellName"].apply(lambda x: cellNameDict[x])
  return data


In [7]:
#Refactoring data from features to useful values
X_train = refactorFeaturesDataframe(X_train).to_numpy()
y_train = y_train.to_numpy()

X_validation = refactorFeaturesDataframe(X_validation).to_numpy()
y_validation = y_validation.to_numpy()

X_test = refactorFeaturesDataframe(test).to_numpy()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Solving the Classification Problem

In [8]:
clf = sklearn.tree.DecisionTreeClassifier(random_state=1)
pipe = Pipeline(steps=[('std_slc', StandardScaler()),
                           ('dec_tree', clf)])

n_components = list(range(1,X_train.shape[1]+1,2))
criterion = ['gini', 'entropy']
max_depth = [None,2,8,12]
min_samples_split = [2,4,8,10]
min_samples_leaf = [1,2,5]

parameters = dict(dec_tree__criterion=criterion,
                      dec_tree__min_samples_split=min_samples_split,
                      dec_tree__min_samples_leaf=min_samples_leaf,
                      dec_tree__max_depth=max_depth)
#clf_GS = GridSearchCV(pipe, parameters)
#clf_GS.fit(X_train, y_train)

clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [9]:
#pred_train = clf_GS.predict(X_train)
#pred_val = clf_GS.predict(X_validation)

# AdaBoost with GridSearch

In [10]:
#ADABOOST
from sklearn.ensemble import AdaBoostClassifier
from joblib import dump, load

adaboost_mode_file = 'anomaly-4G-detection/adaboost_model.joblib'
performVoting = input("Press 'y' to perform Voting Classifier...")
print(performVoting)
if performVoting != 'y':
  try:
    clf_GS = load(a) 
    print(clf_GS)

  except:
    clf = AdaBoostClassifier(random_state=1)
    pipe = Pipeline(steps=[('std_slc', StandardScaler()),
                              ('ada', clf)])
    n_components = list(range(1,X_train.shape[1]+1,2))
    n_estimators = [200, 400, 600]
    learning_rate = [0.1,1,10]
    parameters = dict(  )
    clf_GS = GridSearchCV(pipe, parameters)
    clf_GS.fit(X_train, y_train)
    dump(clf_GS, adaboost_mode_file) 

  pred_train = clf_GS.predict(X_train)
  pred_val = clf_GS.predict(X_validation)


Press 'y' to perform Voting Classifier...y
y


# Voting Classifier with GridSearch

In [198]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

if performVoting == 'y':
  
  try:
    clf_GS = load('anomaly-4G-detection/voting_model.joblib') 
    

  except:
    voting_clf = VotingClassifier( estimators=[ 
        ('xgb', XGBClassifier(random_state=1, objective="binary:logistic", max_depth=15, min_child_weight=1)),
        ('dt', DecisionTreeClassifier(random_state=1, criterion='entropy', min_samples_leaf=2)),
        ('knn', KNeighborsClassifier(metric="euclidean", weights="uniform", p = 2))
        ], voting='soft')
    
    pipe = Pipeline(steps=[('std_slc', StandardScaler()),
                                ('voting_clf', voting_clf)])

    #Params ADA
    n_estimators_ada = [600, 800]
    learning_rate = [0.1,1,10]
    #Params Decision Tree
    min_samples_split_dt = [2,10,14]
    min_samples_leaf_dt = [1,2,5]
    #Params GradientBoost
    parameters = {
     'voting_clf__xgb__eta'    : [0.10, 0.20, 0.30 ] ,
     'voting_clf__xgb__n_estimators' : [ 50, 150, 300],
     'voting_clf__xgb__learning_rate' : [ 0.01, 0.1],
     'voting_clf__xgb__gamma' : [ 0.1, 0.3, 0.5],
     'voting_clf__knn__n_neighbors'    : [15, 19, 22],
     'voting_clf__knn__p'    : [1, 2],
     }


    '''
    parameters = dict(voting_clf__ada__n_estimators=n_estimators_ada,
                      voting_clf__decision_tree__min_samples_split = min_samples_split_dt,
                        )
    '''
    clf_GS = GridSearchCV(estimator=pipe, param_grid=parameters, n_jobs=10, verbose=1, )
    clf_GS.fit(X_train,y_train)
    dump(clf_GS, 'voting_model.joblib') 

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  5.9min
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed: 36.2min
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed: 86.7min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed: 158.6min
[Parallel(n_jobs=10)]: Done 1230 tasks      | elapsed: 251.8min
[Parallel(n_jobs=10)]: Done 1620 out of 1620 | elapsed: 334.2min finished


In [199]:
pred_train = clf_GS.predict(X_train)
pred_val = clf_GS.predict(X_validation)

In [200]:
print("TRAINING\n" + classification_report(y_train, pred_train))
print("\nTESTING\n" + classification_report(y_validation, pred_val))

TRAINING
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     21377
           1       1.00      1.00      1.00      8146

    accuracy                           1.00     29523
   macro avg       1.00      1.00      1.00     29523
weighted avg       1.00      1.00      1.00     29523


TESTING
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5344
           1       0.99      0.93      0.96      2037

    accuracy                           0.98      7381
   macro avg       0.98      0.97      0.97      7381
weighted avg       0.98      0.98      0.98      7381



In [201]:
train_error = 1. - accuracy_score(y_train, pred_train)
train_cmat = confusion_matrix(y_train, pred_train)
val_error = 1. - accuracy_score(y_validation, pred_val)
val_cmat = confusion_matrix(y_validation, pred_val)

print('train error: %f ' % train_error)
print('train confusion matrix:')
print(train_cmat)
print('test error: %f ' % val_error)
print('test confusion matrix:')
print(val_cmat)

train error: 0.000711 
train confusion matrix:
[[21377     0]
 [   21  8125]]
test error: 0.020864 
test confusion matrix:
[[5323   21]
 [ 133 1904]]


In [202]:
pred_test = clf_GS.predict(X_test)

# Submission Formatting

In [203]:
%%shell
# Create submission file if it does not exists
file=predictions.csv
if [ ! -e "$file" ] ; then
    touch anomaly-4G-detection/"$file"
fi

In [204]:
# Create index column in data frame object
submission_dataframe = pd.DataFrame(np.arange(1, 9159), columns=['Id']) 

# Append predictions of test data as column
submission_dataframe['Label'] = pred_test

# Convert Data Frame object to CSV
submission_dataframe.to_csv('predictions.csv', index=False)

!mv predictions.csv anomaly-4G-detection/
predictions = pd.read_csv('anomaly-4G-detection/predictions.csv')
predictions

,Id,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,1
...,...,...
9153,9154,0
9154,9155,1
9155,9156,0
9156,9157,0


In [205]:
#!rm anomaly-4G-detection/predictions.csv

In [206]:
clf_GS.best_params_

{'voting_clf__knn__n_neighbors': 19,
 'voting_clf__knn__p': 1,
 'voting_clf__xgb__eta': 0.1,
 'voting_clf__xgb__gamma': 0.5,
 'voting_clf__xgb__learning_rate': 0.1,
 'voting_clf__xgb__n_estimators': 300}